<a href="https://colab.research.google.com/github/hyunaeee/StableDiffusion/blob/main/face_ai_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U autotrain-advanced

  Using cached diffusers-0.21.4-py3-none-any.whl (1.5 MB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.25.0.dev0
    Uninstalling diffusers-0.25.0.dev0:
      Successfully uninstalled diffusers-0.25.0.dev0


In [ ]:
!autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [28]:
import os

model_name = 'stabilityai/stable-diffusion-xl-base-1.0'
prompt = "photo of a hyunae_test" #Need_to_fix_a_keyword
output_path = "/content/drive/MyDrive/hyunaeee/result"
image_path = "/content/drive/MyDrive/hyunaeee/images" #Need_to_fix_a_keyword

learning_rate = 1e-4
num_steps = 1000
batch_size = 2
gradient_accumulation = 4
resolution = 1024
use_8bit_adam = True
use_xformers = True
use_fp16 = True
train_text_encoder = False
gradient_checkpointing = True

os.environ["MODEL_NAME"] = model_name
os.environ["PROJECT_NAME"] = output_path
os.environ["PROMPT"] = prompt
os.environ["IMAGE_PATH"] = image_path
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_STEPS"] = str(num_steps)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["RESOLUTION"] = str(resolution)
os.environ["USE_8BIT_ADAM"] = str(use_8bit_adam)
os.environ["USE_XFORMERS"] = str(use_xformers)
os.environ["USE_FP16"] = str(use_fp16)
os.environ["GRADIENT_CHECKPOINTING"] = str(gradient_checkpointing)

print(os.listdir(image_path))
print(os.environ.get("IMAGE_PATH"))

['IMG_3543.HEIC', 'IMG_3564.HEIC', 'IMG_3558.HEIC', 'IMG_3556.HEIC', 'IMG_3552.HEIC', 'IMG_3544.HEIC']
/content/drive/MyDrive/hyunaeee/images


In [ ]:
!ls -al /content/drive/MyDrive/hyunaeee

total 11363
drwx------ 2 root root    4096 Dec  4 10:48 images
-rw------- 1 root root 1797419 Dec  4 09:28 IMG_3543.HEIC
-rw------- 1 root root 2353703 Dec  4 09:28 IMG_3544.HEIC
-rw------- 1 root root 1998437 Dec  4 09:28 IMG_3552.HEIC
-rw------- 1 root root 1690467 Dec  4 09:28 IMG_3556.HEIC
-rw------- 1 root root 1907831 Dec  4 09:28 IMG_3558.HEIC
-rw------- 1 root root 1873523 Dec  4 09:28 IMG_3564.HEIC
drwx------ 2 root root    4096 Dec  4 09:30 .ipynb_checkpoints
drwx------ 2 root root    4096 Dec  4 09:30 result


In [30]:
!autotrain dreambooth \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--image-path ${IMAGE_PATH} \
--prompt "${PROMPT}" \
--resolution ${RESOLUTION} \
--batch-size ${BATCH_SIZE} \
--num-steps ${NUM_STEPS} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--lr ${LEARNING_RATE} \
$( [[ "$USE_FP16" == "True" ]] && echo "--fp16" ) \
$( [[ "$USE_XFORMERS" == "True" ]] && echo "--xformers" ) \
$( [[ "$USE_8BIT_ADAM" == "True" ]] && echo "--use-8bit-adam" ) \
$( [[ "$GRADIENT_CHECKPOINTING" == "True" ]] && echo "--gradient-checkpointing" )

> INFO    Namespace(version=False, model='stabilityai/stable-diffusion-xl-base-1.0', revision=None, tokenizer=None, image_path='/content/drive/MyDrive/hyunaeee/images', class_image_path=None, prompt='photo of a hyunae_test', class_prompt=None, num_class_images=100, class_labels_conditioning=None, prior_preservation=None, prior_loss_weight=1.0, project_name='/content/drive/MyDrive/hyunaeee/result', seed=42, resolution=1024, center_crop=None, train_text_encoder=None, batch_size=2, sample_batch_size=4, epochs=1, num_steps=1000, checkpointing_steps=100000, resume_from_checkpoint=None, gradient_accumulation=4, gradient_checkpointing=True, lr=0.0001, scale_lr=None, scheduler='constant', warmup_steps=0, num_cycles=1, lr_power=1.0, dataloader_num_workers=0, use_8bit_adam=True, adam_beta1=0.9, adam_beta2=0.999, adam_weight_decay=0.01, adam_epsilon=1e-08, max_grad_norm=1.0, allow_tf32=None, prior_generation_precision=None, local_rank=-1, xformers=True, pre_compute_text_embeddings=None, tokenizer

In [ ]:
!pip install invisible_watermark transformers accelerate safetensors
!pip install git+https://github.com/huggingface/diffusers
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch
model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
model,
torch_dtype=torch.float16,
)
pipe.to("cuda")
pipe.load_lora_weights("/content/drive/MyDrive/{filename}.safetensors")
refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
"stabilityai/stable-diffusion-xl-refiner-1.0",
torch_dtype=torch.float16,
)
refiner.to("cuda")

In [ ]:
prompt = "A cute woman, photorealistic, bold and elegant, close up, sharp expression, look at camera, block noise, christmas colors, fuzz, abstract background, bold outlines, ultrafine detailed, best quality"
negative_prompt = "ugly, deformed, noisy, blurry, distorted, grainy, text, cropped"
#for seed in range(10):
generator = torch.Generator("cuda").manual_seed(43)
image = pipe(prompt=prompt, negative_prompt=negative_prompt, generator=generator, num_inference_steps=40)
image = image.images[0]
image = refiner(prompt=prompt, negative_prompt=negative_prompt, generator=generator, image=image)
image = image.images[0]